# Model Selection and Regularization

Methods for selecting the best subset of predictors in linear regression.

## Contents
1. Best Subset Selection
2. Model Selection Criteria (RSS, R², Adjusted R², Cp, BIC)
3. Forward and Backward Stepwise Selection
4. Validation Set Approach
5. K-Fold Cross-Validation for Model Selection

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Load Hitters Dataset

Baseball players' salaries and performance statistics.

In [ ]:
# Load Hitters dataset
url = "https://raw.githubusercontent.com/selva86/datasets/master/Hitters.csv"
Hitters = pd.read_csv(url)

print(f"Dataset shape: {Hitters.shape}")
print(f"\nColumn names:")
print(Hitters.columns.tolist())
print(f"\nFirst few rows:")
Hitters.head()

In [ ]:
# Check for missing values
print("Missing values per column:")
print(Hitters.isnull().sum())
print(f"\nTotal missing values in Salary: {Hitters['Salary'].isnull().sum()}")

In [ ]:
# Remove rows with missing Salary
Hitters = Hitters.dropna()

print(f"Dataset shape after removing NAs: {Hitters.shape}")
print(f"Total missing values: {Hitters.isnull().sum().sum()}")

In [ ]:
# Data summary
print("Summary statistics:")
Hitters.describe()

## 1. Best Subset Selection

Try all possible combinations of predictors and select the best model for each size.

In [ ]:
# Prepare data
# Convert categorical variables to dummy variables
X = pd.get_dummies(Hitters.drop('Salary', axis=1), drop_first=True)
y = Hitters['Salary']

print(f"Number of predictors: {X.shape[1]}")
print(f"Predictor names: {X.columns.tolist()}")

In [ ]:
def best_subset_selection(X, y, max_features=None):
    """
    Perform best subset selection.
    
    For each model size k, find the best k-variable model.
    """
    if max_features is None:
        max_features = X.shape[1]
    
    n = len(y)
    results = []
    
    for k in range(1, max_features + 1):
        print(f"\rEvaluating models with {k} predictors...", end='')
        
        best_rss = np.inf
        best_features = None
        best_r2 = -np.inf
        
        # Try all combinations of k features
        for features in combinations(X.columns, k):
            X_subset = X[list(features)]
            model = LinearRegression()
            model.fit(X_subset, y)
            y_pred = model.predict(X_subset)
            
            rss = np.sum((y - y_pred) ** 2)
            r2 = r2_score(y, y_pred)
            
            if rss < best_rss:
                best_rss = rss
                best_features = features
                best_r2 = r2
                best_model = model
        
        # Calculate other metrics
        p = k
        adj_r2 = 1 - (1 - best_r2) * (n - 1) / (n - p - 1)
        
        # Cp (Mallows' Cp)
        mse_full = best_rss / n  # Using current model's MSE as estimate
        cp = (best_rss / mse_full) - n + 2 * p
        
        # BIC
        bic = n * np.log(best_rss / n) + p * np.log(n)
        
        results.append({
            'n_features': k,
            'features': best_features,
            'rss': best_rss,
            'r2': best_r2,
            'adj_r2': adj_r2,
            'cp': cp,
            'bic': bic,
            'model': best_model
        })
    
    print("\nDone!")
    return pd.DataFrame(results)

# Note: For large datasets, this is computationally expensive!
# We'll limit to 8 features for demonstration
print("Running best subset selection (limited to 8 features for speed)...")
best_subset_results = best_subset_selection(X, y, max_features=8)

In [ ]:
# Display results
print("\n" + "="*80)
print("BEST SUBSET SELECTION RESULTS")
print("="*80)
display_cols = ['n_features', 'rss', 'r2', 'adj_r2', 'cp', 'bic']
print(best_subset_results[display_cols].round(4))
print("="*80)

## 2. Model Selection Criteria

Plot different criteria to choose optimal model size.

In [ ]:
# Plot selection criteria
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# RSS
axes[0, 0].plot(best_subset_results['n_features'], best_subset_results['rss'], 
                marker='o', linewidth=2, markersize=6)
axes[0, 0].set_xlabel('Number of Variables')
axes[0, 0].set_ylabel('RSS')
axes[0, 0].set_title('Residual Sum of Squares')
axes[0, 0].grid(True, alpha=0.3)

# Adjusted R²
best_idx_adjr2 = best_subset_results['adj_r2'].idxmax()
axes[0, 1].plot(best_subset_results['n_features'], best_subset_results['adj_r2'], 
                marker='o', linewidth=2, markersize=6)
axes[0, 1].scatter(best_subset_results.loc[best_idx_adjr2, 'n_features'], 
                   best_subset_results.loc[best_idx_adjr2, 'adj_r2'], 
                   color='red', s=200, zorder=5, label=f"Best: {best_subset_results.loc[best_idx_adjr2, 'n_features']} vars")
axes[0, 1].set_xlabel('Number of Variables')
axes[0, 1].set_ylabel('Adjusted R²')
axes[0, 1].set_title('Adjusted R²')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Cp
best_idx_cp = best_subset_results['cp'].idxmin()
axes[1, 0].plot(best_subset_results['n_features'], best_subset_results['cp'], 
                marker='o', linewidth=2, markersize=6)
axes[1, 0].scatter(best_subset_results.loc[best_idx_cp, 'n_features'], 
                   best_subset_results.loc[best_idx_cp, 'cp'], 
                   color='red', s=200, zorder=5, label=f"Best: {best_subset_results.loc[best_idx_cp, 'n_features']} vars")
axes[1, 0].set_xlabel('Number of Variables')
axes[1, 0].set_ylabel('Cp')
axes[1, 0].set_title("Mallows' Cp")
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# BIC
best_idx_bic = best_subset_results['bic'].idxmin()
axes[1, 1].plot(best_subset_results['n_features'], best_subset_results['bic'], 
                marker='o', linewidth=2, markersize=6)
axes[1, 1].scatter(best_subset_results.loc[best_idx_bic, 'n_features'], 
                   best_subset_results.loc[best_idx_bic, 'bic'], 
                   color='red', s=200, zorder=5, label=f"Best: {best_subset_results.loc[best_idx_bic, 'n_features']} vars")
axes[1, 1].set_xlabel('Number of Variables')
axes[1, 1].set_ylabel('BIC')
axes[1, 1].set_title('Bayesian Information Criterion')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBest model by Adjusted R²: {best_subset_results.loc[best_idx_adjr2, 'n_features']} variables")
print(f"Best model by Cp: {best_subset_results.loc[best_idx_cp, 'n_features']} variables")
print(f"Best model by BIC: {best_subset_results.loc[best_idx_bic, 'n_features']} variables")

## 3. Forward and Backward Stepwise Selection

More efficient alternatives to best subset selection.

In [ ]:
def forward_stepwise_selection(X, y, max_features=None):
    """
    Forward stepwise selection: Start with no variables, add one at a time.
    """
    if max_features is None:
        max_features = X.shape[1]
    
    remaining = set(X.columns)
    selected = []
    results = []
    
    for k in range(1, max_features + 1):
        print(f"\rForward step {k}...", end='')
        
        best_rss = np.inf
        best_feature = None
        
        for feature in remaining:
            features = selected + [feature]
            X_subset = X[features]
            model = LinearRegression()
            model.fit(X_subset, y)
            y_pred = model.predict(X_subset)
            rss = np.sum((y - y_pred) ** 2)
            
            if rss < best_rss:
                best_rss = rss
                best_feature = feature
                best_model = model
        
        selected.append(best_feature)
        remaining.remove(best_feature)
        
        # Calculate metrics
        X_subset = X[selected]
        y_pred = best_model.predict(X_subset)
        r2 = r2_score(y, y_pred)
        
        results.append({
            'n_features': k,
            'features': tuple(selected),
            'rss': best_rss,
            'r2': r2
        })
    
    print(" Done!")
    return pd.DataFrame(results)

def backward_stepwise_selection(X, y):
    """
    Backward stepwise selection: Start with all variables, remove one at a time.
    """
    selected = list(X.columns)
    results = []
    
    # Full model
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    rss = np.sum((y - y_pred) ** 2)
    r2 = r2_score(y, y_pred)
    
    results.append({
        'n_features': len(selected),
        'features': tuple(selected),
        'rss': rss,
        'r2': r2
    })
    
    while len(selected) > 1:
        print(f"\rBackward step (removing from {len(selected)} vars)...", end='')
        
        best_rss = np.inf
        worst_feature = None
        
        for feature in selected:
            features = [f for f in selected if f != feature]
            X_subset = X[features]
            model = LinearRegression()
            model.fit(X_subset, y)
            y_pred = model.predict(X_subset)
            rss = np.sum((y - y_pred) ** 2)
            
            if rss < best_rss:
                best_rss = rss
                worst_feature = feature
                best_model = model
        
        selected.remove(worst_feature)
        
        # Calculate metrics
        X_subset = X[selected]
        y_pred = best_model.predict(X_subset)
        r2 = r2_score(y, y_pred)
        
        results.append({
            'n_features': len(selected),
            'features': tuple(selected),
            'rss': best_rss,
            'r2': r2
        })
    
    print(" Done!")
    return pd.DataFrame(results).sort_values('n_features').reset_index(drop=True)

In [ ]:
# Run forward and backward selection
print("Running forward stepwise selection...")
forward_results = forward_stepwise_selection(X, y, max_features=8)

print("\nRunning backward stepwise selection...")
backward_results = backward_stepwise_selection(X)
backward_results = backward_results[backward_results['n_features'] <= 8]

In [ ]:
# Compare methods
print("\n" + "="*80)
print("COMPARISON: BEST SUBSET vs FORWARD vs BACKWARD")
print("="*80)

comparison = pd.DataFrame({
    'n_features': best_subset_results['n_features'],
    'Best_Subset_R2': best_subset_results['r2'],
    'Forward_R2': forward_results['r2'],
    'Backward_R2': backward_results['r2']
})

print(comparison.round(4))
print("="*80)

In [ ]:
# Visualize comparison
plt.figure(figsize=(10, 6))
plt.plot(best_subset_results['n_features'], best_subset_results['r2'], 
         marker='o', label='Best Subset', linewidth=2, markersize=8)
plt.plot(forward_results['n_features'], forward_results['r2'], 
         marker='s', label='Forward', linewidth=2, markersize=8)
plt.plot(backward_results['n_features'], backward_results['r2'], 
         marker='^', label='Backward', linewidth=2, markersize=8)
plt.xlabel('Number of Variables')
plt.ylabel('R²')
plt.title('Comparison of Selection Methods')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Validation Set Approach

In [ ]:
# Split data
np.random.seed(1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

In [ ]:
# Forward selection on training set
forward_train_results = forward_stepwise_selection(X_train, y_train, max_features=X.shape[1])

# Evaluate on test set
val_errors = []

for idx, row in forward_train_results.iterrows():
    features = list(row['features'])
    X_test_subset = X_test[features]
    
    # Refit model on training data with these features
    model = LinearRegression()
    model.fit(X_train[features], y_train)
    
    # Predict on test set
    y_pred = model.predict(X_test_subset)
    mse = mean_squared_error(y_test, y_pred)
    val_errors.append(mse)

forward_train_results['test_mse'] = val_errors

In [ ]:
# Plot validation errors
plt.figure(figsize=(10, 6))
plt.plot(forward_train_results['n_features'], forward_train_results['test_mse'], 
         marker='o', linewidth=2, markersize=8)

best_idx = forward_train_results['test_mse'].idxmin()
plt.scatter(forward_train_results.loc[best_idx, 'n_features'], 
           forward_train_results.loc[best_idx, 'test_mse'], 
           color='red', s=200, zorder=5, 
           label=f"Best: {forward_train_results.loc[best_idx, 'n_features']} variables")

plt.xlabel('Number of Variables')
plt.ylabel('Test MSE')
plt.title('Validation Set Approach')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nBest model has {forward_train_results.loc[best_idx, 'n_features']} variables")
print(f"Test MSE: {forward_train_results.loc[best_idx, 'test_mse']:.4f}")
print(f"\nFeatures: {forward_train_results.loc[best_idx, 'features']}")

## 5. K-Fold Cross-Validation for Model Selection

In [ ]:
# K-Fold Cross-Validation
k = 10
np.random.seed(1)
kf = KFold(n_splits=k, shuffle=True, random_state=1)

# Matrix to store CV errors
cv_errors = np.zeros((k, X.shape[1]))

for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\rProcessing fold {fold_idx + 1}/{k}...", end='')
    
    X_train_fold = X.iloc[train_idx]
    y_train_fold = y.iloc[train_idx]
    X_test_fold = X.iloc[test_idx]
    y_test_fold = y.iloc[test_idx]
    
    # Forward selection on this fold
    fold_results = forward_stepwise_selection(X_train_fold, y_train_fold, max_features=X.shape[1])
    
    # Evaluate each model size
    for idx, row in fold_results.iterrows():
        features = list(row['features'])
        
        # Refit and predict
        model = LinearRegression()
        model.fit(X_train_fold[features], y_train_fold)
        y_pred = model.predict(X_test_fold[features])
        mse = mean_squared_error(y_test_fold, y_pred)
        
        cv_errors[fold_idx, row['n_features'] - 1] = mse

print(" Done!")

# Calculate mean CV error for each model size
mean_cv_errors = cv_errors.mean(axis=0)
std_cv_errors = cv_errors.std(axis=0)

In [ ]:
# Plot CV errors
plt.figure(figsize=(10, 6))
plt.plot(range(1, X.shape[1] + 1), mean_cv_errors, marker='o', linewidth=2, markersize=8)
plt.fill_between(range(1, X.shape[1] + 1), 
                 mean_cv_errors - std_cv_errors,
                 mean_cv_errors + std_cv_errors,
                 alpha=0.2)

best_idx_cv = np.argmin(mean_cv_errors)
plt.scatter(best_idx_cv + 1, mean_cv_errors[best_idx_cv], 
           color='red', s=200, zorder=5, label=f"Best: {best_idx_cv + 1} variables")

plt.xlabel('Number of Variables')
plt.ylabel('Cross-Validation MSE')
plt.title(f'{k}-Fold Cross-Validation')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nBest model by {k}-Fold CV: {best_idx_cv + 1} variables")
print(f"CV MSE: {mean_cv_errors[best_idx_cv]:.4f} ± {std_cv_errors[best_idx_cv]:.4f}")

In [ ]:
# Fit final model on full dataset with optimal number of features
final_results = forward_stepwise_selection(X, y, max_features=X.shape[1])
best_features = list(final_results.iloc[best_idx_cv]['features'])

print(f"\nFinal model with {len(best_features)} variables:")
print(best_features)

# Fit and display coefficients
final_model = LinearRegression()
final_model.fit(X[best_features], y)

coef_df = pd.DataFrame({
    'Feature': best_features,
    'Coefficient': final_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("\nCoefficients:")
print(coef_df)
print(f"\nIntercept: {final_model.intercept_:.4f}")

## Summary

This notebook covered:

### **Best Subset Selection**
- Try all possible combinations of predictors
- Computationally expensive: 2^p models for p predictors
- Guarantees finding the best model for each size

### **Forward Stepwise Selection**
- Start with no variables, add one at a time
- Greedy algorithm: may not find global optimum
- Much faster: only p(p+1)/2 models

### **Backward Stepwise Selection**
- Start with all variables, remove one at a time
- Also greedy
- Requires n > p (more observations than predictors)

### **Selection Criteria**
- **RSS**: Always decreases with more variables
- **R²**: Always increases with more variables
- **Adjusted R²**: Penalizes model complexity
- **Cp**: Estimates test error
- **BIC**: Stronger penalty than Cp

### **Cross-Validation**
- Most reliable method for model selection
- Directly estimates test error
- K=5 or K=10 are common choices

### **Key Takeaways:**
- More variables ≠ better model (overfitting!)
- Use CV to select model size
- Forward/backward are good approximations to best subset
- Different criteria may suggest different model sizes